In [1]:
import numpy as np

def apply_pbc(coords, lattice_vectors):
    """
    Apply periodic boundary conditions to bring coordinates inside the unit cell.

    Parameters:
    coords (numpy.ndarray): The coordinates of the atoms, shape (N, 3).
    lattice_vectors (numpy.ndarray): The lattice vectors, shape (3, 3).

    Returns:
    numpy.ndarray: The coordinates wrapped inside the unit cell.
    """
    # Convert coordinates to fractional coordinates
    fractional_coords = np.linalg.solve(lattice_vectors.T, coords.T).T

    # Apply periodic boundary conditions: bring fractional coordinates into the range [0, 1)
    fractional_coords -= np.floor(fractional_coords)

    # Convert back to cartesian coordinates
    wrapped_coords = fractional_coords.dot(lattice_vectors)

    return wrapped_coords

# Example usage
coords = np.array([
    [3.5, 4.5, 5.5],  # Example coordinates that might be outside the unit cell
    [-0.5, 1.5, 2.5],
    [6.0, 6.0, 6.0]
])

lattice_vectors = np.array([
    [1.0, 0.0, 0.0],  # Lattice vector a
    [0.0, 1.0, 0.0],  # Lattice vector b
    [0.0, 0.0, 1.0]   # Lattice vector c
])

wrapped_coords = apply_pbc(coords, lattice_vectors)
print("Original coordinates:")
print(coords)
print("Wrapped coordinates:")
print(wrapped_coords)

Original coordinates:
[[ 3.5  4.5  5.5]
 [-0.5  1.5  2.5]
 [ 6.   6.   6. ]]
Wrapped coordinates:
[[0.5 0.5 0.5]
 [0.5 0.5 0.5]
 [0.  0.  0. ]]


In [2]:
import torch
def apply_pbc(coords, lattice_vectors):
    """
    Apply periodic boundary conditions to bring coordinates inside the unit cell.

    Parameters:
    coords (torch.Tensor): The coordinates of the atoms, shape (N, 3).
    lattice_vectors (torch.Tensor): The lattice vectors, shape (3, 3).

    Returns:
    torch.Tensor: The coordinates wrapped inside the unit cell.
    """
    # Convert coordinates to fractional coordinates
    inv_lattice_vectors = torch.linalg.inv(lattice_vectors)
    fractional_coords = torch.matmul(coords, inv_lattice_vectors.T)

    # Apply periodic boundary conditions: bring fractional coordinates into the range [0, 1)
    fractional_coords = fractional_coords - torch.floor(fractional_coords)

    # Convert back to cartesian coordinates
    wrapped_coords = torch.matmul(fractional_coords, lattice_vectors)

    return wrapped_coords

# Example usage
coords = torch.tensor([
    [3.5, 4.5, 5.5],  # Example coordinates that might be outside the unit cell
    [-0.5, 1.5, 2.5],
    [6.0, 6.0, 6.0]
])

lattice_vectors = torch.tensor([
    [2.0, 0.0, 0.0],  # Lattice vector a
    [0.0, 2.0, 0.0],  # Lattice vector b
    [0.0, 0.0, 2.0]   # Lattice vector c
])

wrapped_coords = apply_pbc(coords, lattice_vectors)
print("Original coordinates:")
print(coords)
print("Wrapped coordinates:")
print(wrapped_coords)

: 

In [2]:
import ase.io
data = ase.io.read("../IONS+CENTERS_cell_test.xyz")

In [3]:
import cpmd.asign_wcs_torch

: 